In [1]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-sample.json
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-labels.pkl
# !wget https://f000.backblazeb2.com/file/malay-dataset/voxceleb/voxceleb2-test-wav.tar

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import malaya_speech
malaya_speech

`pyaudio` is not available, `malaya_speech.streaming.stream` is not able to use.


<module 'malaya_speech' from '/home/husein/dev/malaya-speech/malaya_speech/__init__.py'>

In [4]:
model = malaya_speech.speaker_vector.nemo('huseinzol05/nemo-titanet_large')

In [5]:
model.eval()

Model(
  (preprocessor): AudioToMelSpectrogramPreprocessor(
    (featurizer): FilterbankFeatures()
  )
  (encoder): ConvASREncoder(
    (encoder): Sequential(
      (0): JasperBlock(
        (mconv): ModuleList(
          (0): MaskedConv1d(
            (conv): Conv1d(80, 80, kernel_size=(3,), stride=(1,), padding=(1,), groups=80, bias=False)
          )
          (1): MaskedConv1d(
            (conv): Conv1d(80, 1024, kernel_size=(1,), stride=(1,), bias=False)
          )
          (2): BatchNorm1d(1024, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (3): SqueezeExcite(
            (fc): Sequential(
              (0): Linear(in_features=1024, out_features=128, bias=False)
              (1): ReLU(inplace=True)
              (2): Linear(in_features=128, out_features=1024, bias=False)
            )
            (gap): AdaptiveAvgPool1d(output_size=1)
          )
        )
        (mout): Sequential(
          (0): ReLU(inplace=True)
          (1): Dropout(p=0.0, 

In [6]:
_ = model.cuda()

In [7]:
import json
import pickle
from tqdm import tqdm

In [8]:
with open('voxceleb2-test-sample.json') as fopen:
    sample_files = json.load(fopen)

In [9]:
with open('voxceleb2-test-labels.pkl', 'rb') as fopen:
    labels = pickle.load(fopen)

In [10]:
unique_files = []
for l in labels:
    unique_files.extend(l[1:])
    
unique_files = list(set(unique_files))

In [11]:
len(unique_files)

36237

In [12]:
import torch
import numpy as np

In [13]:
vectors = {}
for f in tqdm(unique_files):
    y_, _ = malaya_speech.load(f.replace('/home/husein/youtube/', '/home/husein/'))
    try:
        v = model([y_])[0]
        vectors[f] = v
    except:
        torch.cuda.empty_cache()

100%|█████████████████████████████████████| 36237/36237 [07:55<00:00, 76.16it/s]


In [14]:
import numpy as np

scores, ls = [], []

for i in tqdm(range(len(labels))):
    if labels[i][1] in vectors and labels[i][2] in vectors:
        ls.append(labels[i][0])
        scores.append(np.sum(vectors[labels[i][1]] * vectors[labels[i][2]]))

100%|█████████████████████████████| 5900000/5900000 [00:15<00:00, 378323.25it/s]


In [15]:
len(scores), len(vectors)

(5900000, 36237)

In [16]:
def calculate_eer(y, y_score):
    
    from scipy.optimize import brentq
    from sklearn.metrics import roc_curve
    from scipy.interpolate import interp1d

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [17]:
calculate_eer(ls, scores)

(0.022779999999967302, array(0.01879263))

In [18]:
v.shape

(192,)